# Python Pandas Notes

## Sorting

In [ ]:
# Decreasing order
coffee.sort_values("Units Sold", ascending=False)
coffee.sort_values(["Day"], ascending=[0])

## Filtering Data

In [ ]:
bios.info()
bios.loc[bios["height_cm"] > 215, ["name", "height_cm"]]
bios[(bios["height_cm"] > 215) & (bios["born_country"] == "USA")]
bios[bios["name"].str.contains("Keith", case=True)]
bios.query("born_country == 'USA' and biosnicity == 'White'")

## Adding & Removing Columns

In [ ]:
coffee["price"] = 4.99
import numpy as np
coffee["new_price"] = np.where(coffee["coffee_type"] == "Espresso", 3.99, 5.99)
coffee.drop(0)
coffee.drop(columns=["price"])
coffee.drop(columns=["price"], inplace=True)
coffee = coffee[["Day", "coffee_type"]]
coffee["revenue"] = coffee["Units Sold"] * coffee["new_price"]
coffee.rename(columns={"new_price": "Price"}, inplace=True)
coffee.head()

## String Operations

In [ ]:
bios_new = bios.copy()
bios_new["first_name"] = bios_new["name"].str.split(" ").str[0]
bios_new.query("first_name == 'Keith'")
bios.head()

## DateTime Operations

In [ ]:
bios_new["born_datetime"] = pd.to_datetime(bios_new["born_date"])
bios_new["born_year"] = bios_new["born_datetime"].dt.year
bios_new[["name", "born_year"]]

## Saving DataFrame

In [ ]:
bios_new.to_csv("data/bios-new.csv", index=False)

## Custom Columns using Lambda and Apply

In [ ]:
bios["height_category"] = bios["height_cm"].apply(
    lambda x: "Short" if x < 165 else ("Average" if x < 185 else "Tall"))

In [ ]:
def categorize_athlete(row):
    if row["height_cm"] > 195 and row["weight_kg"] < 75:
        return "Lightweight"
    elif row["height_cm"] < 185 or row["weight_kg"] <= 80:
        return "Midweight"
    else:
        return "Heavyweight"

bios["category"] = bios.apply(categorize_athlete, axis=1)
bios.head()

## Advanced Functionality

In [ ]:
coffee["yesterday_revenue"] = coffee["revenue"].shift(1)
bios["height_rank"] = bios["height_cm"].rank()
bios.sort_values("height_rank", ascending=False)
coffee["cumulative_revenue"] = coffee["revenue"].cumsum()
latte = coffee[coffee["coffee_type"] == "latte"].copy()
latte["3day_sum"] = latte["Units Sold"].rolling(3).sum()

## Handling Null Values

In [ ]:
coffee.loc[0, "Units Sold"] = np.nan
coffee = coffee.fillna(0)
coffee.fillna(coffee["Units Sold"].mean(), inplace=True)
coffee.interpolate(inplace=True)
coffee.dropna()
coffee.dropna(subset=["Units Sold"], inplace=True)
coffee[coffee["Units Sold"].isna()]

## Aggregating Data

In [ ]:
coffee.groupby("coffee_type")["Units Sold"].mean()
pivot = coffee.pivot(columns="coffee_type", index="Day", values="revenue")
pivot.sum()
pivot.sum(axis=1)

## Merging and Concatenating Data

In [ ]:
nos = pd.read_csv("data/noc-regions.csv")
bios_new = pd.merge(bios, nos, left_on="born_country", right_on="NOC", how="left")
bios_new.rename(columns={"region": "born_country_full"}, inplace=True)
combined_df = pd.merge(results, bios, on="athlete_id", how="left")

### Concatenation

In [ ]:
usa = bios[bios["born_country"] == "USA"].copy()
gbr = bios[bios["born_country"] == "GBR"].copy()
new_df = pd.concat([usa, gbr])